# PIK Digital Day

### Utilities

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook

In [2]:
def rmse(a, b):
    return mean_squared_error(a, b)**0.5

### Load Data

In [3]:
train = pd.read_csv("train.csv", encoding="cp1251")
test = pd.read_csv("test.csv", encoding="cp1251")
test_id = test["id"]

train = train.drop(['vid_2', 'plan_s', 'plan_l', 'vid_0', 'plan_m', 'start_square', 'vid_1'], axis=1)

train["is_train"] = True
test["is_train"] = False

data = pd.concat((train, test))
data = data.drop("date1", axis=1)

/home/alxmamaev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Remove the CWD from sys.path while we load stuff.


### Preprocess Data

In [4]:
data.head(3)

,Cтавка по ипотеке,bulk_id,id,is_train,mean_fl,mean_sq,month,month_cnt,price,spalen,...,Площадь зеленой зоны в радиусе 500 м,Площадь земельного участка,Площадь пром. зоны в радиусе 500 м,Подземная парковка,Поликлиника,Система мусоротведения,Спортивная площадка,Станций метро от кольца,ФОК,Школа
0,10.04,FF3814A9-9F7D-E711-8530-00505688958B,0,True,8,38.00,10,34,135633.913043,1,...,25533,17.0,65357,да,0,контейнеры,да,8,0,825
1,10.04,FF3814A9-9F7D-E711-8530-00505688958B,1,True,9,58.01,10,34,128492.419825,2,...,25533,17.0,65357,да,0,контейнеры,да,8,0,825
2,10.04,FF3814A9-9F7D-E711-8530-00505688958B,2,True,10,22.58,10,34,158237.634409,0,...,25533,17.0,65357,да,0,контейнеры,да,8,0,825


In [5]:
categorical_features = []
for i in data.drop("bulk_id", axis=1).columns:
    if data[i].dtype =="object":
        categorical_features.append(i)

categorical_features

['Автомойка',
 'Входные группы',
 'Двор без машин',
 'Кладовые',
 'Класс объекта',
 'Колясочные',
 'Огорожена территория',
 'Подземная парковка',
 'Система мусоротведения',
 'Спортивная площадка']

In [6]:
for cat in categorical_features:
    data[cat] = pd.factorize(data[cat])[0]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11061 entries, 0 to 1816
Data columns (total 49 columns):
Cтавка по ипотеке                        11061 non-null float64
bulk_id                                  11061 non-null object
id                                       11061 non-null int64
is_train                                 11061 non-null bool
mean_fl                                  11061 non-null int64
mean_sq                                  11061 non-null float64
month                                    11061 non-null int64
month_cnt                                11061 non-null int64
price                                    11061 non-null float64
spalen                                   11061 non-null int64
value                                    9244 non-null float64
Автомойка                                11061 non-null int64
Вентлияция                               11061 non-null int64
Видеонаблюдение                          11061 non-null int64
Вклады до 1 года 

In [8]:
flat = pd.read_csv("flat.csv", encoding="cp1251")

Множество за период по корпусу/комнатности (далее Сет)- подможество из Множества за период по проекту  состоящее из квартир с одинаковой комнатностью(spalen) и из одного корпуса (bulk). Предсказание делается для каждого такого множества.

In [9]:
flat_real_features = ["floor", "balcon", "square", "Количество помещений"]
flat_cat_features = ["vid", "otdelka", "Класс объекта",
                    "Огорожена территория", "Кондиционирование", "Вентлияция",
                    "Система мусоротведения"]

for bulk_id in tqdm_notebook(data["bulk_id"].unique()):
    set_spalen = data[data["bulk_id"] == bulk_id]["spalen"].unique()
    for spalen in set_spalen:
        data_indexes = (data["bulk_id"] == bulk_id) & (data["spalen"] == spalen)
        flat_indexes = (flat["bulk_id"] == bulk_id) & (flat["spalen"] == spalen)
        
        for f in flat_real_features:
            data.loc[data_indexes, "max_"+f] = flat[flat_indexes][f].max()
            data.loc[data_indexes, "min_"+f] = flat[flat_indexes][f].min()
            
            data.loc[data_indexes, "mean_"+f] = flat[flat_indexes][f].mean()
            data.loc[data_indexes, "median_"+f] = flat[flat_indexes][f].median()
            
            data.loc[data_indexes, "variance_"+f] = ((flat[flat_indexes][f] - flat[flat_indexes][f].mean())**2).mean()
            data.loc[data_indexes, "std_"+f] = ((flat[flat_indexes][f] \
                                                 - flat[flat_indexes][f].mean())**2).mean() ** 0.5
        for f in flat_cat_features:
            for c in flat[f].unique():
                flat_indexes2 = (flat_indexes & (flat[f] == c))
                data.loc[data_indexes, f+"_"+str(c)+"_count"] = flat_indexes2.sum()
                data.loc[data_indexes, f+"_"+str(c)+"_share_object"] = flat_indexes2.sum()/flat_indexes.sum()

In [11]:
data["Средняя цена за квадрат"] = data["price"]/data["mean_sq"]
data["Средняя цена за комнату"] = data["price"]/(data["spalen"]+1)
data["Средняя площадь комнаты"] = data["mean_sq"]/(data["spalen"]+1)
data["Средняя площадь комнаты"] = data["mean_sq"]/(data["spalen"]+1)

data["Удобство до большой дороги"] = data["До большой дороги на машине(км)"]/data["Машиномест"]
data["Удобство до авторазвязки"] = data["До удобной авторазвязки на машине(км)"]/data["Машиномест"]
data["Удобство до парка на машине"] = data["До парка(км)"]/data["Машиномест"]

data["Удобство до кремля пешком"] = data["До Кремля"]/data["До метро пешком(км)"]
data["Удобство до парка пешком"] = data["До парка(км)"]/data["До метро пешком(км)"]
data["Критерий удобства Метро"] = data["Станций метро от кольца"]/data["До метро пешком(км)"]
data["price_USD"] = data["price"]/data["Курс"]

data["Общая сумма вкладов"] = data["Вклады от 1 года до 3 лет"] + data['Вклады свыше 3 лет'] + data['Вклады до 1 года']
data["Соотношение вкладов до 1 и после 1"] = data['Вклады до 1 года'] / (data["Вклады от 1 года до 3 лет"] + data['Вклады свыше 3 лет'])
data["Соотношение вкладов до 3 и после 3"] = (data['Вклады до 1 года'] + data["Вклады от 1 года до 3 лет"]) / data['Вклады свыше 3 лет']

In [12]:
data = data.drop(["id", "bulk_id"], axis=1)
data = data.fillna(-1000)
train = data[data["is_train"] == True]
test = data[data["is_train"] == False]

### Train model

In [25]:
n_folds = 10
max_depth = 10
iterations = 3200

In [26]:
kf = KFold(n_splits=n_folds, random_state=42, shuffle=True)

In [27]:
for i, (train_index, valid_index) in enumerate(kf.split(range(len(train)))):
    train_fold = train.loc[train_index]
    valid_fold = train.loc[valid_index]

    train_fold.to_csv(f"./input/train_{i}.csv",index=False)
    valid_fold.to_csv(f"./input/valid_{i}.csv",index=False)

In [28]:
models_predict = []
feature_importances = []
models_rmse = []

for i in tqdm(range(n_folds)):
    train_fold = pd.read_csv(f"./input/train_{i}.csv")
    valid_fold = pd.read_csv(f"./input/valid_{i}.csv")
    
    x_train, y_train = train_fold.drop("value", axis=1), train_fold["value"]
    x_val, y_val = valid_fold.drop("value", axis=1), valid_fold["value"]
    x_test = test.drop("value", axis=1)
    
    model = CatBoostRegressor(iterations=iterations, 
                              learning_rate=0.05, 
                              depth=max_depth, 
                              random_seed = 42)
    
    model.fit(x_train, y_train, eval_set=(x_val, y_val))#, cat_features=cat_cols_indexes)
    
    models_rmse.append(rmse(model.predict(x_val), y_val))
    pred = model.predict(x_test)
    models_predict.append(pred)
    feature_importances.append(model.get_feature_importance())


  0%|          | 0/10 [00:00<?, ?it/s]

0:	learn: 436.2030333	test: 417.9238035	best: 417.9238035 (0)	total: 84.7ms	remaining: 4m 31s
1:	learn: 426.3017979	test: 407.4987957	best: 407.4987957 (1)	total: 163ms	remaining: 4m 20s
2:	learn: 416.6277383	test: 397.8831062	best: 397.8831062 (2)	total: 248ms	remaining: 4m 23s
3:	learn: 408.0463475	test: 388.9831787	best: 388.9831787 (3)	total: 333ms	remaining: 4m 26s
4:	learn: 399.3595791	test: 380.1986058	best: 380.1986058 (4)	total: 414ms	remaining: 4m 24s
5:	learn: 391.3048903	test: 372.2498163	best: 372.2498163 (5)	total: 495ms	remaining: 4m 23s
6:	learn: 384.0201895	test: 364.9261179	best: 364.9261179 (6)	total: 594ms	remaining: 4m 30s
7:	learn: 376.5193209	test: 357.6463385	best: 357.6463385 (7)	total: 679ms	remaining: 4m 30s
8:	learn: 369.9380822	test: 351.2386687	best: 351.2386687 (8)	total: 756ms	remaining: 4m 28s
9:	learn: 363.8839174	test: 345.2101576	best: 345.2101576 (9)	total: 840ms	remaining: 4m 28s
10:	learn: 358.0375996	test: 338.6699845	best: 338.6699845 (10)	total

87:	learn: 236.3769997	test: 239.9022448	best: 239.9022448 (87)	total: 6.41s	remaining: 3m 46s
88:	learn: 235.8926385	test: 239.7604490	best: 239.7604490 (88)	total: 6.48s	remaining: 3m 46s
89:	learn: 235.2956236	test: 239.5940995	best: 239.5940995 (89)	total: 6.55s	remaining: 3m 46s
90:	learn: 234.6970517	test: 239.2663041	best: 239.2663041 (90)	total: 6.62s	remaining: 3m 46s
91:	learn: 234.0602745	test: 238.8856210	best: 238.8856210 (91)	total: 6.69s	remaining: 3m 46s
92:	learn: 233.5294462	test: 238.5902994	best: 238.5902994 (92)	total: 6.76s	remaining: 3m 45s
93:	learn: 232.8410427	test: 238.2374271	best: 238.2374271 (93)	total: 6.83s	remaining: 3m 45s
94:	learn: 232.2116388	test: 237.9374443	best: 237.9374443 (94)	total: 6.9s	remaining: 3m 45s
95:	learn: 231.5946422	test: 237.5921567	best: 237.5921567 (95)	total: 6.97s	remaining: 3m 45s
96:	learn: 231.1646660	test: 237.3002961	best: 237.3002961 (96)	total: 7.05s	remaining: 3m 45s
97:	learn: 230.6902856	test: 237.1421833	best: 237.

174:	learn: 201.8377475	test: 225.1369735	best: 225.1369735 (174)	total: 12.5s	remaining: 3m 36s
175:	learn: 201.5744486	test: 225.1106943	best: 225.1106943 (175)	total: 12.6s	remaining: 3m 36s
176:	learn: 201.2745181	test: 225.0294655	best: 225.0294655 (176)	total: 12.6s	remaining: 3m 36s
177:	learn: 200.9129077	test: 224.8519784	best: 224.8519784 (177)	total: 12.7s	remaining: 3m 36s
178:	learn: 200.6693348	test: 224.9504780	best: 224.8519784 (177)	total: 12.8s	remaining: 3m 35s
179:	learn: 200.5538045	test: 224.9168949	best: 224.8519784 (177)	total: 12.9s	remaining: 3m 35s
180:	learn: 200.2958741	test: 224.7506455	best: 224.7506455 (180)	total: 12.9s	remaining: 3m 35s
181:	learn: 200.0724941	test: 224.6385960	best: 224.6385960 (181)	total: 13s	remaining: 3m 35s
182:	learn: 199.8976892	test: 224.6377079	best: 224.6377079 (182)	total: 13.1s	remaining: 3m 35s
183:	learn: 199.6311533	test: 224.6660093	best: 224.6377079 (182)	total: 13.1s	remaining: 3m 35s
184:	learn: 199.4795788	test: 22

261:	learn: 180.6331485	test: 216.6391295	best: 216.6391295 (261)	total: 18.6s	remaining: 3m 28s
262:	learn: 180.4353720	test: 216.5306487	best: 216.5306487 (262)	total: 18.7s	remaining: 3m 28s
263:	learn: 180.3009228	test: 216.6062116	best: 216.5306487 (262)	total: 18.7s	remaining: 3m 28s
264:	learn: 180.0540978	test: 216.4995290	best: 216.4995290 (264)	total: 18.8s	remaining: 3m 28s
265:	learn: 179.9443386	test: 216.4225726	best: 216.4225726 (265)	total: 18.9s	remaining: 3m 28s
266:	learn: 179.7297377	test: 216.3670414	best: 216.3670414 (266)	total: 18.9s	remaining: 3m 28s
267:	learn: 179.4478185	test: 216.3325146	best: 216.3325146 (267)	total: 19s	remaining: 3m 28s
268:	learn: 179.2740168	test: 216.2846642	best: 216.2846642 (268)	total: 19.1s	remaining: 3m 28s
269:	learn: 178.9531894	test: 216.0917608	best: 216.0917608 (269)	total: 19.2s	remaining: 3m 27s
270:	learn: 178.7067723	test: 215.9765496	best: 215.9765496 (270)	total: 19.2s	remaining: 3m 27s
271:	learn: 178.6166861	test: 21

...

2890:	learn: 82.7844758	test: 215.7740022	best: 215.7365240 (2781)	total: 3m 18s	remaining: 21.2s
2891:	learn: 82.7825459	test: 215.7782887	best: 215.7365240 (2781)	total: 3m 18s	remaining: 21.1s
2892:	learn: 82.7819729	test: 215.7776225	best: 215.7365240 (2781)	total: 3m 18s	remaining: 21s
2893:	learn: 82.7816813	test: 215.7795537	best: 215.7365240 (2781)	total: 3m 18s	remaining: 21s
2894:	learn: 82.7809314	test: 215.7859173	best: 215.7365240 (2781)	total: 3m 18s	remaining: 20.9s
2895:	learn: 82.7407475	test: 215.7799670	best: 215.7365240 (2781)	total: 3m 18s	remaining: 20.8s
2896:	learn: 82.7280274	test: 215.7832206	best: 215.7365240 (2781)	total: 3m 18s	remaining: 20.8s
2897:	learn: 82.7243373	test: 215.7799123	best: 215.7365240 (2781)	total: 3m 18s	remaining: 20.7s
2898:	learn: 82.7157492	test: 215.7743527	best: 215.7365240 (2781)	total: 3m 18s	remaining: 20.6s
2899:	learn: 82.7149148	test: 215.7745886	best: 215.7365240 (2781)	total: 3m 18s	remaining: 20.6s
2900:	learn: 82.7025696	

2974:	learn: 81.9250706	test: 215.5881757	best: 215.5881757 (2974)	total: 3m 23s	remaining: 15.4s
2975:	learn: 81.9238759	test: 215.5968652	best: 215.5881757 (2974)	total: 3m 23s	remaining: 15.3s
2976:	learn: 81.8853375	test: 215.5943826	best: 215.5881757 (2974)	total: 3m 23s	remaining: 15.3s
2977:	learn: 81.8843576	test: 215.5934824	best: 215.5881757 (2974)	total: 3m 23s	remaining: 15.2s
2978:	learn: 81.8596546	test: 215.6245347	best: 215.5881757 (2974)	total: 3m 24s	remaining: 15.1s
2979:	learn: 81.8552716	test: 215.6292261	best: 215.5881757 (2974)	total: 3m 24s	remaining: 15.1s
2980:	learn: 81.8397064	test: 215.6302446	best: 215.5881757 (2974)	total: 3m 24s	remaining: 15s
2981:	learn: 81.8194390	test: 215.6264406	best: 215.5881757 (2974)	total: 3m 24s	remaining: 14.9s
2982:	learn: 81.7795655	test: 215.6287678	best: 215.5881757 (2974)	total: 3m 24s	remaining: 14.9s
2983:	learn: 81.7779121	test: 215.6279282	best: 215.5881757 (2974)	total: 3m 24s	remaining: 14.8s
2984:	learn: 81.764435

3059:	learn: 81.0694770	test: 215.5214069	best: 215.5115645 (3044)	total: 3m 29s	remaining: 9.58s
3060:	learn: 81.0531301	test: 215.5383722	best: 215.5115645 (3044)	total: 3m 29s	remaining: 9.52s
3061:	learn: 81.0461468	test: 215.5391638	best: 215.5115645 (3044)	total: 3m 29s	remaining: 9.45s
3062:	learn: 81.0375826	test: 215.5419038	best: 215.5115645 (3044)	total: 3m 29s	remaining: 9.38s
3063:	learn: 81.0242847	test: 215.5554586	best: 215.5115645 (3044)	total: 3m 29s	remaining: 9.31s
3064:	learn: 81.0196320	test: 215.5540154	best: 215.5115645 (3044)	total: 3m 29s	remaining: 9.24s
3065:	learn: 80.9581671	test: 215.4997656	best: 215.4997656 (3065)	total: 3m 29s	remaining: 9.17s
3066:	learn: 80.9462493	test: 215.4983570	best: 215.4983570 (3066)	total: 3m 29s	remaining: 9.1s
3067:	learn: 80.9450736	test: 215.4971322	best: 215.4971322 (3067)	total: 3m 30s	remaining: 9.04s
3068:	learn: 80.9139917	test: 215.5102925	best: 215.4971322 (3067)	total: 3m 30s	remaining: 8.97s
3069:	learn: 80.90968

3143:	learn: 80.1957867	test: 215.4885552	best: 215.4885552 (3143)	total: 3m 35s	remaining: 3.83s
3144:	learn: 80.1813854	test: 215.4828651	best: 215.4828651 (3144)	total: 3m 35s	remaining: 3.76s
3145:	learn: 80.1630279	test: 215.4656844	best: 215.4656844 (3145)	total: 3m 35s	remaining: 3.7s
3146:	learn: 80.1552134	test: 215.4542505	best: 215.4542505 (3146)	total: 3m 35s	remaining: 3.63s
3147:	learn: 80.1216530	test: 215.4625882	best: 215.4542505 (3146)	total: 3m 35s	remaining: 3.56s
3148:	learn: 80.1161739	test: 215.4609274	best: 215.4542505 (3146)	total: 3m 35s	remaining: 3.49s
3149:	learn: 80.1031603	test: 215.4629505	best: 215.4542505 (3146)	total: 3m 35s	remaining: 3.42s
3150:	learn: 80.0574494	test: 215.4487183	best: 215.4487183 (3150)	total: 3m 35s	remaining: 3.35s
3151:	learn: 80.0540394	test: 215.4445142	best: 215.4445142 (3151)	total: 3m 35s	remaining: 3.29s
3152:	learn: 80.0515118	test: 215.4461018	best: 215.4445142 (3151)	total: 3m 35s	remaining: 3.22s
3153:	learn: 80.03793


100%|██████████| 10/10 [40:25<00:00, 242.53s/it]


In [29]:
print("Mean RMSE:", np.mean(models_rmse))

Mean RMSE: 207.56751447165553


In [30]:
feature_importances = np.mean(feature_importances[1:], axis=0)
columns = train.drop("value", axis=1).columns

sorted(list(zip(feature_importances, columns)))[::-1]

[(5.146096838415528, 'mean_fl'),
 (4.842559613516093, 'Система мусоротведения_контейнеры_count'),
 (4.676782760332619, 'Огорожена территория_нет_count'),
 (4.663652707367763, 'price_USD'),
 (4.123568632068799, 'vid_хороший_count'),
 (4.066854128879646, 'Cтавка по ипотеке'),
 (3.97797602093022, 'month_cnt'),
 (3.358779307836268, 'Класс объекта_комфорт_count'),
 (3.1380086643439205, 'price'),
 (3.1176303433205117, 'month'),
 (2.3148217473688444, 'Средняя цена за комнату'),
 (2.121219694822846, 'Средняя площадь комнаты'),
 (1.892557037663939, 'Вклады от 1 года до 3 лет'),
 (1.7962119004275803, 'otdelka_Не производится_share_object'),
 (1.7846221379641496, 'Удобство до парка пешком'),
 (1.6982138842543262, 'Кондиционирование_1_count'),
 (1.6785785427734567, 'mean_sq'),
 (1.6398196294486265, 'Соотношение вкладов до 3 и после 3'),
 (1.555809297435896, 'Средняя цена за квадрат'),
 (1.53215742022181, 'Площадь двора'),
 (1.4498375717145005, 'variance_square'),
 (1.430883959873894, 'otdelka_Не п

In [31]:
predict = np.mean(models_predict, axis=0)
predict[predict<0] = 0
test["value"] = predict

/home/alxmamaev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
submit = pd.DataFrame()
submit["id"] = test_id
submit["value"] = test["value"]
submit.to_csv("submit.csv", index=False)